In [2]:
import cv2
import supervision as sv
from roboflow import Roboflow

def main():
    # --- 設定 ---
    # 1. 將 'YOUR_API_KEY' 替換成您的 Roboflow Private API Key
    # 2. 您可以在 Roboflow Universe 頁面上找到 project_id 和 version_number
    API_KEY = "UCLBeCClmaD7BW6BWuLG"
    PROJECT_ID = "potato-detection-3et6q"
    MODEL_VERSION = 11  # 您可以更改為模型頁面上顯示的最新版本號

    # --- 2. 初始化 Roboflow 模型 ---
    # 程式會自動使用您環境中已設定好的 GPU
    try:
        rf = Roboflow(api_key=API_KEY)
        project = rf.workspace().project(PROJECT_ID)
        model = project.version(MODEL_VERSION).model
        print("✅ 模型載入成功！正在使用 GPU 進行加速...")
    except Exception as e:
        print(f"❌ 模型載入失敗: {e}")
        return

    # --- 3. 開啟 Webcam ---
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("❌ 無法開啟攝影機。")
        return

    print("🚀 Webcam 已開啟，按下 'q' 鍵結束程式。")

    # --- 4. 即時偵測迴圈 ---
    while True:
        # 讀取影像幀
        ret, frame = cap.read()
        if not ret:
            print("❌ 無法讀取影像幀。")
            break

        # 使用 Roboflow 模型進行預測
        results = model.predict(frame, confidence=40, overlap=30).json()

        # 將 Roboflow 的預測結果轉換為 Supervision 的 Detections 物件
        detections = sv.Detections.from_inference(results)

        # --- 5. 使用 supervision 0.26.1 語法進行繪圖 ---
        # 複製一幀影像用於繪圖
        annotated_frame = frame.copy()
        
        # 遍歷所有偵測到的物件
        for i in range(len(detections)):
            # 從 detections 物件中取得邊界框座標 (xyxy格式)
            box = detections.xyxy[i]
            
            # 從原始的 results 字典中取得類別名稱和信心度
            prediction_info = results['predictions'][i]
            class_name = prediction_info['class']
            confidence = prediction_info['confidence']
            
            # 組合要顯示的標籤文字
            label = f"{class_name} {confidence:.2f}"
            
            # 使用 0.26.1 版本的繪圖工具來畫出方框和標籤
            sv.draw.bounding_box(scene=annotated_frame, xyxy=box)
            sv.draw.label(scene=annotated_frame, text=label, anchor=sv.Point(x=box[0], y=box[1]))
        
        # 顯示結果影像
        cv2.imshow("Potato Detection (GPU) - Press 'q' to exit", annotated_frame)

        # 按下 'q' 鍵即可退出
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # --- 6. 清理資源 ---
    cap.release()
    cv2.destroyAllWindows()
    print("👋 程式已結束。")

if __name__ == "__main__":
    main()


loading Roboflow workspace...
loading Roboflow project...
✅ 模型載入成功！正在使用 GPU 進行加速...
🚀 Webcam 已開啟，按下 'q' 鍵結束程式。


AttributeError: module 'supervision.draw' has no attribute 'bounding_box'